In [ ]:
import warnings
warnings.filterwarnings("ignore")
import os, sys
#sys.path.insert(0,"/home/suman/Sayari/TrP-cage/Notebooks_Trpcage") # File directory
#import modules
import numpy as np
import MDAnalysis as mda
from sklearn.preprocessing import MinMaxScaler
import pyemma
import matplotlib.pyplot as plt
##custom scripts
from runner import Feature
from plot2d import plot_free_energy as pfe

In [ ]:
plt.style.use("Sayari/TrP-cage/style.txt") # give path to this style.txt file

def Ticker(ax):
    """
    Styles the matplotlib axis..

    """
    from matplotlib.ticker import AutoMinorLocator
    minor_locator = AutoMinorLocator(2)
    ax.xaxis.set_minor_locator(minor_locator)
    minor_locator = AutoMinorLocator(2)
    ax.yaxis.set_minor_locator(minor_locator)
    return ax

In [8]:
## De Shaw TRP-Cage 208 micro-sec trajectory
path = "/home/suman/Dibyendu/Trp-Cage-folding/DESRES-Trajectory_2JOF-0-protein/2JOF-0-protein/"
files = os.listdir(path)
traj_list = []
for file in files:
    if file.endswith(".dcd"):
        traj_list.append(path + file)

sorted_traj_list = list(np.sort(traj_list))
trp_cage = mda.Universe("/home/suman/Sayari/TrP-cage/trp_cage.pdb", sorted_traj_list )

In [ ]:
## Features...give the important contact pairs that have discriminative powers..
#If you used other features calculate them accordingly and then scale them

In [9]:
contacts = np.loadtxt('contacts_CA.txt').astype(int) #.txt file of contact pairs of our interest
feats = Feature()
feats.contacts = np.array(contacts)
feats.prepare_contacts(mode="distance")
feats.feature_extraction(mda_universe=trp_cage)
ca_dist=feats.features
np.savetxt("pairwise_dist.txt", ca_dist)

In [10]:
## Scaling of data..
scaler = MinMaxScaler()
scaler.fit(ca_dist)
scaled_dist = scaler.transform(ca_dist)

# VAE

In [5]:
from VAE import VAE
from sklearn.model_selection import train_test_split
import torch

In [ ]:
#torch.save(model.state_dict(),"/home/suman/Sayari/TrP-cage/Trp_cage_VAE.pt")

In [ ]:
# Train Model
model =  VAE (input_dim=190, num_layers=2, nodes=[1024, 512], code = 2, beta=1.0,batch_size=64,epochs=100,thresh=0.5,learning_rate=0.0001,activation="GELU")
model.load_state_dict(torch.load("path/Trp_cage_VAE.pt")) # Give correct path to .pt file
model.eval()

In [11]:
model.eval().to('cpu')
mu_trp_cage = model.encode(torch.Tensor(scaled_dist))[1].detach().numpy()

# TICA

In [19]:
Distances_tica=ca_dist.reshape(1,1044000,190)
tica = pyemma.coordinates.tica(list(Distances_tica), lag=250) #lagtime 50ns

calculate covariances:   0%|                                                                      | 0/6 [00:00…

In [20]:
tica_output = tica.get_output()
tica_concatenated = np.concatenate(tica_output)

getting output of TICA:   0%|                                                                     | 0/6 [00:00…

# PCA

In [24]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
pca.fit(scaled_dist)
X_train_pca = pca.transform(scaled_dist)

# Projection Comparison

In [ ]:
fig, axs = plt.subplots(1, 3, sharex=False, sharey=False,
                         gridspec_kw={'hspace': 0.01},figsize=[26,6])
#Ticker(ax
#Ticker(ax)
#line_labels = ["Monomer", "Pentamer-ChainA", "Pentamer-ChainB", "Pentamer-ChainC","Pentamer-ChainD","Pentamer-ChainE"]
(ax1, ax2,ax3)= axs
Ticker(ax1)
_, _, misc = pfe(X_train_pca[:,0][::][:], X_train_pca[:,1][::],ax=ax1, zorder = 10,cbar=True, legacy = False)
misc['cbar'].set_label('Free Energy/kT', fontsize = 20, color = "darkred", fontweight = "bold")
ax1.set_title("PCA Projection",fontsize=30,loc="center")
#ax1.set_ylabel(r"$PC_2$",fontsize=20)
ax1.set_xlim(-2.6,3.8)
ax1.set_ylim(-2.5,3.8)
ax1.set_xlabel("PC1",fontsize=30)
ax1.set_ylabel("PC2",fontsize=30)

Ticker(ax2)
_, _, misc = pfe(tica_concatenated[:,0][::][:], tica_concatenated[:,1][::],ax=ax2, zorder = 10,cbar=True, legacy = False)
misc['cbar'].set_label('Free Energy/kT', fontsize = 20, color = "darkred", fontweight = "bold")

ax2.set_title("TICA Projection",fontsize=30,loc="center")
ax2.set_ylim(-2.3,3)
ax2.set_xlim(-2.6,1.2)
ax2.set_xlabel("IC1",fontsize=30)
ax2.set_ylabel("IC2",fontsize=30)

Ticker(ax3)
_, _, misc = pfe(mu_trp_cage[:,0][::][:], mu_trp_cage[:,1][::],ax=ax3, zorder = 10,cbar=True, legacy = False)
misc['cbar'].set_label('Free Energy/kT', fontsize = 20, color = "darkred", fontweight = "bold")
ax3.set_title("VAE Projection",fontsize=30,loc="center")
ax3.set_ylim(-2.0,2)
ax3.set_xlim(-2.6,1.8)
ax3.set_xlabel(r"$\mu_1$",fontsize=30)
ax3.set_ylabel(r"$\mu_2$",fontsize=30)
fig.tight_layout()